# Programming Task: Linear Regression

In [1]:
import numpy as np

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

## Your task

This notebook provides a code skeleton for performing linear regression. 
Your task is to complete the functions where required. 
You are only allowed to use built-in Python functions, as well as any `numpy` functions. No other libraries / imports are allowed.

In the beginning of every function there is docstring which specifies the input and and expected output.
Write your code in a way that adheres to it.
You may only use plain python and anything that we imported for you above such as numpy functions (i.e. no scikit-learn classifiers).

## Load and preprocess the data

In this assignment we will work with the Boston Housing Dataset.
The data consists of 506 samples. Each sample represents a district in the city of Boston and has 13 features, such as crime rate or taxation level. The regression target is the median house price in the given district (in $1000's).

More details can be found here: http://lib.stat.cmu.edu/datasets/boston

In [2]:
X , y = fetch_california_housing(return_X_y=True)

# Add a vector of ones to the data matrix to absorb the bias term
# (Recall slide #7 from the lecture)
X = np.hstack([np.ones([X.shape[0], 1]), X])
"""
X 假设是一个二维数组（或称为矩阵），其中每一行代表一个观察值，每一列代表一个特征。

X.shape[0] 获取X的行数，即观察值的数量。

np.ones([X.shape[0], 1]) 创建一个形状为 (X.shape[0], 1) 的数组，这个数组的每一个元素都是1。这实际上创建了一个列向量，其中包含与X中的行数相同数量的1，通常这一列被称作截距项(intercept)或偏置项(bias)。

np.hstack([...]) 将创建的全1列向量和原始的X数组水平堆叠起来。这意味着全1的列被添加到X的左边，作为第一列。
"""
# From now on, D refers to the number of features in the AUGMENTED dataset (i.e. including the dummy '1' feature for the absorbed bias term)

# Split into train and test
test_size = 0.9
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

## Task 1: Fit standard linear regression

In [4]:
def fit_least_squares(X, y):
    """Fit ordinary least squares model to the data.
    
    Parameters
    ----------
    X : array, shape [N, D]
        (Augmented) feature matrix.
    y : array, shape [N]
        Regression targets.
        
    Returns
    -------
    w : array, shape [D]
        Optimal regression coefficients (w[0] is the bias term).
        
    """
    ### BEGIN SOLUTION ###
    """
    np.linalg.pinv(X) 计算矩阵 X 的伪逆（Moore-Penrose逆）。在普通最小二乘法中，当 X 的列数（特征数）小于行数（样本数）并且 X 的列向量（特征）线性无关时，可以使用标准的矩阵逆（np.linalg.inv(X.T @ X) @ X.T）来计算系数。但是，当 X 不是满秩的或者特征数大于样本数时，标准的矩阵逆不存在，这时候使用伪逆是更安全的选择，因为它总是存在。

@ 是Python中的矩阵乘法运算符。它将 X 的伪逆矩阵与目标向量 y 相乘。
np.linalg.pinv(X) @ y 的结果是系数向量 w，该向量最小化了残差平方和，即最小化了 
在这里，w[0]（如果 X 被增广了的话）将是截距项，也就是偏置项。
    """
    return np.linalg.pinv(X) @ y
    ### END SOLUTION ###

## Task 2: Fit ridge regression

In [5]:
def fit_ridge(X, y, reg_strength):
    """Fit ridge regression model to the data.
    
    Parameters
    ----------
    X : array, shape [N, D]
        (Augmented) feature matrix.
    y : array, shape [N]
        Regression targets.
    reg_strength : float
        L2 regularization strength (denoted by lambda in the lecture)
        
    Returns
    -------
    w : array, shape [D]
        Optimal regression coefficients (w[0] is the bias term).
    
    """
    ### BEGIN SOLUTION ###
    return np.linalg.inv(X.T @ X + reg_strength * np.eye(X.shape[1])) @ X.T @ y
    ### END SOLUTION ###

## Task 3: Generate predictions for new data

In [6]:
def predict_linear_model(X, w):
    """Generate predictions for the given samples.
    
    Parameters
    ----------
    X : array, shape [N, D]
        (Augmented) feature matrix.
    w : array, shape [D]
        Regression coefficients.
        
    Returns
    -------
    y_pred : array, shape [N]
        Predicted regression targets for the input data.
        
    """
    ### BEGIN SOLUTION ###
    return X @ w
    ### END SOLUTION ###

## Task 4: Mean squared error

In [7]:
def mean_squared_error(y_true, y_pred):
    """Compute mean squared error between true and predicted regression targets.
    
    Reference: `https://en.wikipedia.org/wiki/Mean_squared_error`
    
    Parameters
    ----------
    y_true : array
        True regression targets.
    y_pred : array
        Predicted regression targets.
        
    Returns
    -------
    mse : float
        Mean squared error.
        
    """
    ### BEGIN SOLUTION ###
    return np.mean((y_true - y_pred)**2)
    ### END SOLUTION ###

## Compare the two models

The reference implementation produces 
* MSE for Least squares $\approx$ **0.5347** , mean squared error 平方后要除以N

* MSE for Ridge regression $\approx$ **0.5331**

You results might be slightly (i.e. $\pm 1\%$) different from the reference soultion due to numerical reasons.

In [8]:
# Load the data
np.random.seed(1234)
X , y = fetch_california_housing(return_X_y=True)
X = np.hstack([np.ones([X.shape[0], 1]), X])
test_size = 0.9 # we select a relatively large test set due to the large size of the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

# Ordinary least squares regression
w_ls = fit_least_squares(X_train, y_train)
y_pred_ls = predict_linear_model(X_test, w_ls)
mse_ls = mean_squared_error(y_test, y_pred_ls)
print('MSE for Least squares = {0}'.format(mse_ls))

# Ridge regression
reg_strength = 1e-2
w_ridge = fit_ridge(X_train, y_train, reg_strength)
y_pred_ridge = predict_linear_model(X_test, w_ridge)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
"""
说明validation set中用了regularization 的误差更小
"""
print('MSE for Ridge regression = {0}'.format(mse_ridge))

MSE for Least squares = 0.5347102426013359
MSE for Ridge regression = 0.5331285867787068
